In [21]:
# -*- coding: utf-8 -*-
"""Outlook folder reader:
 Read specified folder and save items in the folder to csv

original:
https://www.laurivan.com/python-and-outlook-an-example/
see:
https://docs.python.jp/3/using/windows.html#pywin32
http://timgolden.me.uk/pywin32-docs/contents.html
http://timgolden.me.uk/pywin32-docs/html/com/win32com/HTML/QuickStartClientCom.html
http://fum125.hatenablog.com/entry/2016/11/12/233457
https://msdn.microsoft.com/ja-jp/VBA/Outlook-VBA/articles/recipient-type-property-outlook
https://docs.python.jp/3/library/csv.html?highlight=csv
https://msdn.microsoft.com/ja-jp/vba/outlook-vba/articles/propertyaccessor-object-outlook
"""
import win32com.client

DEBUG = False

# Outlook オブジェクト作成

In [22]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# サポートメソッド

In [23]:
def items(outlook_items_object):
    array_size = outlook_items_object.Count
    for item_index in range(1,array_size+1):
        yield (item_index, outlook_items_object[item_index])
    return

        
def prop(outlook_object):
    return sorted( outlook_object._prop_map_get_.keys() )



# フォルダーのパース

In [24]:
roots = outlook.Folders

In [25]:
def parse_folders(folders, depth=0, recursive=True):
    if not folders:
        return
    for index, folder in items(folders):
        print( " "*depth*2, u"(%i) [%s] [%s]" % (index, folder.Name, folder))
        if recursive:
            parse_flders(folder.Folders, depth + 1, recursive)
    return



In [26]:
#parseFolders(roots, recursive=True)

# 特定のフォルダのアイテム表示

In [27]:
def search(folders, query_path):
    if DEBUG:
        print(query_path)
    if not folders:
        return None
    if len(query_path)<=0:
        return None
    for index, folder in items(folders):
        if folder.Name == query_path[0]:
            if len(query_path)==1:
                return folder
            else:
                return search(folder.Folders, query_path[1:])
    return None


def print_items(mail_items):
    for index, mail_item in items(mail_items):
        mail_from = mail_item.SenderEmailAddress + ";"
        mail_to = ""
        mail_cc = ""
        for i, recipient in items(mail_item.Recipients):
            if recipient.Type == win32com.client.constants.olTo:
                mail_to += recipient.Address + ";"
            if (recipient.Type == win32com.client.constants.olCC
                or recipient.Type == win32com.client.constants.olBCC):
                mail_cc += recipient.Address + ";"
            if recipient.Type == win32com.client.constants.olOriginator:
                mail_from += recipient.Address + ";"
        mail_subj = mailItem.Subject
        mail_sentdate = mailItem.SentOn
        print("[%s]\t[%s]\t[%s]\t[%s]\t[%s]"
              % (mail_from, mail_to, mail_cc, mail_subj, mail_sentdate))
    return



In [28]:
folder = search(roots, 'Outlook データ ファイル\\受信トレイ'.split('\\'))
print(folder.Name)
print_items(folder.Items)


受信トレイ


# 特定のフォルダのアイテムをCSV保存

In [29]:
import csv

def get_mail_item_header(mail_item):
    prop_name = "http://schemas.microsoft.com/mapi/proptag/0x007D001E"
    return mail_item.PropertyAccessor.GetProperty(prop_name)


def save_items(mail_items, save_filename):
    with open(save_filename, 'w', newline='', encoding='utf-8', errors='replace') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["from", "to", "cc", "sent date"
                         , "subject", "body", "internet mail header"])

        for index, mail_item in items(mail_items):
            mail_from = mail_item.SenderEmailAddress + ";"
            mail_to = ""
            mail_cc = ""
            for i, recipient in items(mail_item.Recipients):
                if recipient.Type == win32com.client.constants.olTo:
                    mail_to += recipient.Address + ";"
                if (recipient.Type == win32com.client.constants.olCC
                    or recipient.Type == win32com.client.constants.olBCC):
                    mail_cc += recipient.Address + ";"
                if recipient.Type == win32com.client.constants.olOriginator:
                    mail_from += recipient.Address + ";"
            mail_subj = mail_item.Subject
            mail_sentdate = mail_item.SentOn
            mail_body = mail_item.Body
            mail_header = get_mail_item_header(mail_item)
            writer.writerow([mail_from, mail_to, mail_cc, mail_sentdate
                             , mail_subj, mail_body, mail_header])
    return



In [30]:
folder = search(roots, 'Outlook データ ファイル\\受信トレイ\\YMCA'.split('\\'))
print(folder.Name)
save_items(folder.Items, 'test.csv')

YMCA
